# Function definition, run once

In [12]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      %matplotlib
import os
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import csv 

logFolder = r'Z:\Apparatus\DR2\Cooldown Data'
logFolder = r'/home/jaseung/jaseung_office/Labshare/Apparatus/DR2/Cooldown Data'
logFolder = r'/home/jaseung/Downloads/logfiles'

def loadTemperature_oneday(date):
    """
    Return two numpy array of datetime, and temperatures in 'date'
    """
    basePath = os.path.join(logFolder, date)
    file_name = ['CH1 T ' + date + '.log', 'CH2 T ' + date + '.log', 'CH5 T ' + date + '.log', 'CH6 T ' + date + '.log' ]

    fullfile_name = [ os.path.join(basePath, s) for s in file_name ]

    temperatures_all = []
    for f in fullfile_name:
        dateTimes, temperatures = [], []
        fid = open(f, 'r')
        with fid:
            reader = csv.reader(fid, delimiter=",")
            for row in reader:
                dateTime = datetime.strptime(row[0]+row[1], ' %d-%m-%y%H:%M:%S')
                temperature = float(row[2])

                dateTimes.append(dateTime)
                temperatures.append(temperature)

        temperatures_all.append(temperatures)    

    return np.asarray(dateTimes), np.transpose(np.asarray(temperatures_all))

def loadPressure_oneday(date):
    """
    Return two numpy array of datetime, and pressures
    """
    basePath = os.path.join(logFolder, date)
    file_name = 'maxigauge ' + date + '.log'
    full_file_name = os.path.join(basePath, file_name)

    dateTimes,pressures = [], []
    f = open(full_file_name, 'r')
    with f:
        reader = csv.reader(f, delimiter=",")
        for row in reader:
            dateTime = datetime.strptime(row[0]+row[1], '%d-%m-%y%H:%M:%S')

            pressure = [float(row[5]),float(row[11]),float(row[17]),float(row[23]),float(row[29]),float(row[35])] 

            dateTimes.append(dateTime)
            pressures.append(pressure)

    return np.asarray(dateTimes), np.asarray(pressures)

def loadFlowmeter_oneday(date):
    """
    Return two numpy array of datetime, and flowrate
    """
    basePath = os.path.join(logFolder, date)
    file_name = 'Flowmeter ' + date + '.log'
    full_file_name = os.path.join(basePath, file_name)

    dateTimes,flowrates = [], []
    f = open(full_file_name, 'r')
    with f:
        reader = csv.reader(f, delimiter=",")
        for row in reader:
            dateTime = datetime.strptime(row[0]+row[1], ' %d-%m-%y%H:%M:%S')

            flowrate =float(row[2])

            dateTimes.append(dateTime)
            flowrates.append(flowrate)

    return np.asarray(dateTimes), np.asarray(flowrates)

def existingDateFolder(start_date, end_date):
    """
    Returns a list of folder names available between start_date and end_date.
    """
    start_Date = datetime.strptime(start_date, '%y-%m-%d')
    if end_date:
        end_Date = datetime.strptime(end_date, '%y-%m-%d')
    else:
        end_Date = start_Date
    dt = timedelta(days=1)
    dates = np.arange(start_Date, end_Date+dt, dt).astype(datetime)

    dateFolder = [date.strftime('%y-%m-%d') for date in dates]
    dateFolder = [ s for s in dateFolder if os.path.isdir(os.path.join(logFolder, s))]

    return dateFolder

def loadTemperature(start_date, end_date=None):
    """
    Return time and temperature numpy arrays between start_date and end_date
    """
    dateTimes_all, temperatures_all = [], []
    for i, date in enumerate(existingDateFolder(start_date, end_date)):

        dateTimes, temperatures = loadTemperature_oneday(date)

        if i ==0 :
            temperatures_all = temperatures
        else :
            temperatures_all = np.vstack((temperatures_all, temperatures))

        dateTimes_all = np.append(dateTimes_all, dateTimes)
    return dateTimes_all, temperatures_all

def loadPressure(start_date, end_date=None):
    dateTimes_all, pressures_all = [], []
    for i, date in enumerate(existingDateFolder(start_date, end_date)):

        dateTimes, pressures = loadPressure_oneday(date)

        if i ==0 :
            pressures_all = pressures
        else :
            pressures_all = np.vstack((pressures_all, pressures))

        dateTimes_all = np.append(dateTimes_all, dateTimes)
    return dateTimes_all, pressures_all

def loadFlowmeter(start_date, end_date=None):
    dateTimes_all, flowmeters_all = [], []
    for i, date in enumerate(existingDateFolder(start_date, end_date)):

        dateTimes, flowmeters = loadFlowmeter_oneday(date)

#         if i ==0 :
#             flowmeters_all = flowmeters
#         else :
#             flowmeters_all = np.vstack((flowmeters_all, flowmeters))

        dateTimes_all = np.append(dateTimes_all, dateTimes)
        flowmeters_all = np.append(flowmeters_all, flowmeters)
    return dateTimes_all, flowmeters_all

def plotTemperature(start_date, end_date=None):
    """
    Plot temperature date.
    """
    dateTimes_all, temperatures_all = loadTemperature(start_date, end_date)
    plt.figure()
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%y-%m-%d,%H:%M'))
    plt.plot(dateTimes_all, temperatures_all)
    plt.legend(['50K','4K','Still','MC'])
    plt.xlabel('Time')
    plt.ylabel('Temperature (K)')
    if end_date:
        plt.title('Temperature' + ' @' + start_date + ' - ' + end_date)    
    else :
        plt.title('Temperature' + ' @' + start_date)    
    plt.grid('on')

def plotPressure(start_date, end_date=None):
    
    dateTimes_all, pressures_all = loadPressure(start_date, end_date)
    plt.figure()
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%y-%m-%d,%H:%M'))
    plt.plot(dateTimes_all, pressures_all)
    plt.legend(['P1','P2','P3','P4','P5','P6'])
    plt.xlabel('Time')
    plt.ylabel('Pressure (mBar)')
    if end_date:
        plt.title('Pressure' + ' @' + start_date + ' - ' + end_date)    
    else :
        plt.title('Pressure' + ' @' + start_date)    
    plt.grid('on')

def plotFlowmeter(start_date, end_date=None):
    
    dateTimes_all, flowmeters_all = loadFlowmeter(start_date, end_date)
    plt.figure()
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%y-%m-%d,%H:%M'))
    plt.plot(dateTimes_all, flowmeters_all)
    plt.xlabel('Time')
    plt.ylabel('Flowrate (mmol/s)')
    if end_date:
        plt.title('Flowmeter' + ' @' + start_date + ' - ' + end_date)    
    else :
        plt.title('Flowmeter' + ' @' + start_date)   
    plt.grid('on')
    
def plotTempPress(start_date, end_date=None):
    dateTimes_temp_all, temperatures_all = loadTemperature(start_date, end_date)
    dateTimes_press_all, pressures_all = loadPressure(start_date, end_date)
    
    plt.subplot(211)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%y-%m-%d,%H:%M'))
    plt.plot(dateTimes_temp_all, temperatures_all)
    plt.xlabel('Time')
    plt.ylabel('Temperature (K)')
    if end_date:
        plt.title('Temperature' + ' @' + start_date + ' - ' + end_date)    
    else :
        plt.title('Temperature' + ' @' + start_date)    
    plt.grid('on')
    
    plt.subplot(212)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%y-%m-%d,%H:%M'))
    plt.plot(dateTimes_press_all, pressures_all)
    plt.xlabel('Time')
    plt.ylabel('Pressure (mBar)')
    if end_date:
        plt.title('Pressure' + ' @' + start_date + ' - ' + end_date)    
    else :
        plt.title('Pressure' + ' @' + start_date)  

Using matplotlib backend: Qt5Agg


In [105]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, date, timedelta
import textwrap

# logFolder = r'Z:\Apparatus\DR2\Cooldown Data'
logFolder = r'/home/jaseung/Downloads/logfiles'

def loadTemperature_oneday(date:date):
    """
        Read temperature log files and return two pandas dataframe of datetime and temperatures.
    """
    
    date_str = date.strftime("%y-%m-%d")
    base_path = os.path.join(logFolder, date_str)    
    file_names = ['CH1 T ' + date_str + '.log', 
                  'CH2 T ' + date_str + '.log', 
                  'CH5 T ' + date_str + '.log', 
                  'CH6 T ' + date_str + '.log' ]

    full_file_names = [ os.path.join(base_path, file_name) for file_name in file_names ]

    df_datetimes, df_temperatures = pd.DataFrame(), pd.DataFrame()
    
    for file_name in full_file_names:
        with open(file_name, 'r') as f: 
            df = pd.read_csv(f, names=['date', 'time', 'temperature'], header=0)
            df_datetime  = pd.to_datetime(df['date'] + df['time'], format=' %d-%m-%y%H:%M:%S')
            df_datetimes = pd.concat([df_datetimes, df_datetime], axis=1)             
            df_temperatures = pd.concat([df_temperatures, df['temperature']], axis=1)
           
#     print(df_datetimes.shape)
#     print(df_temperatures.shape)   
    
    return df_datetimes, df_temperatures

def loadPressure_oneday(date:date):
    """
    Return two numpy array of datetime, and pressures
    """
    date_str = date.strftime("%y-%m-%d")
    base_path = os.path.join(logFolder, date_str)  
    file_name = 'maxigauge ' + date_str + '.log'
    full_file_name = os.path.join(base_path, file_name)

    df_datetimes, df_pressures = pd.DataFrame(), pd.DataFrame()
    with open(full_file_name, 'r') as f:
        df = pd.read_csv(f, header=None)
        df_datetimes  = pd.to_datetime(df.iloc[:,0] + df.iloc[:,1], format='%d-%m-%y%H:%M:%S')
        # df_datetimes = pd.concat([df_datetimes, df_datetime], axis=1)             
        df_pressures = pd.concat([df_pressures, df.iloc[:,5]], axis=1)
        df_pressures = pd.concat([df_pressures, df.iloc[:,11]], axis=1)
        df_pressures = pd.concat([df_pressures, df.iloc[:,17]], axis=1)
        df_pressures = pd.concat([df_pressures, df.iloc[:,23]], axis=1)
        df_pressures = pd.concat([df_pressures, df.iloc[:,29]], axis=1)
        df_pressures = pd.concat([df_pressures, df.iloc[:,35]], axis=1)

    # print(df_datetimes.shape)
    # print(df_pressures.shape)   

    return df_datetimes, df_pressures

def loadFlowmeter_oneday(date):
    """
    Return two numpy array of datetime, and flowrate
    """
    date_str = date.strftime("%y-%m-%d")
    base_path = os.path.join(logFolder, date_str)  
    file_name = 'Flowmeter ' + date_str + '.log'
    full_file_name = os.path.join(base_path, file_name)

    df_datetimes, df_flowmeter = pd.DataFrame(), pd.DataFrame()
    with open(full_file_name, 'r') as f:
        df = pd.read_csv(f, header=None)
        df_datetimes  = pd.to_datetime(df.iloc[:,0] + df.iloc[:,1], format=' %d-%m-%y%H:%M:%S')
        df_flowmeter = df.iloc[:,2]
       
    # print(df_datetimes.shape)
    # print(df_flowmeter.shape)   

    return df_datetimes, df_flowmeter

def loadStatus_oneday(date:date):
    """
    Return two numpy array of datetime, and pressures
    """
    date_str = date.strftime("%y-%m-%d")
    base_path = os.path.join(logFolder, date_str)  
    file_name = 'Status_' + date_str + '.log'
    full_file_name = os.path.join(base_path, file_name)

    df_datetimes, df_status = pd.DataFrame(), pd.DataFrame()
    with open(full_file_name, 'r') as f:
        df = pd.read_csv(f, header=None)
        df_datetimes  = pd.to_datetime(df.iloc[:,0] + df.iloc[:,1], format='%d-%m-%y%H:%M:%S')
        # df_datetimes = pd.concat([df_datetimes, df_datetime], axis=1)             
        df_status = pd.concat([df_status, df.iloc[:,5]], axis=1)
        df_status = pd.concat([df_status, df.iloc[:,11]], axis=1)
        df_status = pd.concat([df_status, df.iloc[:,17]], axis=1)
        df_status = pd.concat([df_status, df.iloc[:,23]], axis=1)
        df_status = pd.concat([df_status, df.iloc[:,29]], axis=1)
        df_status = pd.concat([df_status, df.iloc[:,35]], axis=1)

    # print(df_datetimes.shape)
    # print(df_pressures.shape)   

    return df_datetimes, df_status


def loadTemperature(start_date:date, end_date:date):
    """
    Return time and temperature numpy arrays between start_date and end_date
    """
    df_datetimes_all, df_temperatures_all = pd.DataFrame(), pd.DataFrame()
    
    while start_date <= end_date:
        
        try:
            df_datetimes, df_temperatures = loadTemperature_oneday(start_date)
        except FileNotFoundError:
            pass
            
        df_datetimes_all = pd.concat([df_datetimes_all, df_datetimes], axis=0)
        df_temperatures_all = pd.concat([df_temperatures_all, df_temperatures], axis=0)
        
        start_date += timedelta(days=1)
    
    return df_datetimes_all, df_temperatures_all
        
def loadPressure(start_date:date, end_date:date):
    df_datetimes_all, df_pressures_all = pd.DataFrame(), pd.DataFrame()
    while start_date <= end_date:
        try:
            df_datetimes, df_pressures = loadPressure_oneday(start_date)
        except FileNotFoundError:
            pass
            
        df_datetimes_all = pd.concat([df_datetimes_all, df_datetimes], axis=0)
        df_pressures_all = pd.concat([df_pressures_all, df_pressures], axis=0)
        
        start_date += timedelta(days=1)
    
    return df_datetimes_all, df_pressures_all

def loadFlowmeter(start_date, end_date):
    df_datetimes_all, df_flowmeters_all = pd.DataFrame(), pd.DataFrame()
    while start_date <= end_date:
        try:
            df_datetimes, df_flowmeters = loadFlowmeter_oneday(start_date)
        except FileNotFoundError:
            pass
            
        df_datetimes_all = pd.concat([df_datetimes_all, df_datetimes], axis=0)
        df_flowmeters_all = pd.concat([df_flowmeters_all, df_flowmeters], axis=0)
        
        start_date += timedelta(days=1)
    
    return df_datetimes_all, df_flowmeters_all

def plotTemperature(start_date, end_date=None):
    """
    Plot temperature date.
    """
    df_df_datetimes_all, df_temperatures_all = loadTemperature(start_date, end_date)

    fig, ax = plt.subplots()
    
    text = "Temperature from " + start_date.strftime("%y-%m-%d") + " to " + end_date.strftime("%y-%m-%d")
    title = "Click on legend line to toggle line on/off" + "\n" + "\n".join(textwrap.wrap(text, 60))
    ax.set_title(title)
    ax.set_xlabel('Datetime')
    ax.set_ylabel('Temperature(K)')
    ax.grid()
    
    line1, = ax.plot(df_df_datetimes_all.iloc[:,0], df_temperatures_all.iloc[:,0], label="50 K")
    line2, = ax.plot(df_df_datetimes_all.iloc[:,1], df_temperatures_all.iloc[:,1], label="4 K")
    line3, = ax.plot(df_df_datetimes_all.iloc[:,2], df_temperatures_all.iloc[:,2], label="Still")
    line4, = ax.plot(df_df_datetimes_all.iloc[:,3], df_temperatures_all.iloc[:,3], label="MCX")
    leg = ax.legend(fancybox=True, shadow=True)
    
    lines = [line1, line2, line3, line4]
    lined = {}  # Will map legend lines to original lines.
    for legline, origline in zip(leg.get_lines(), lines):
        legline.set_picker(True)  # Enable picking on the legend line.
        lined[legline] = origline
    
    def on_pick(event):
        # On the pick event, find the original line corresponding to the legend
        # proxy line, and toggle its visibility.
        legline = event.artist
        origline = lined[legline]
        visible = not origline.get_visible()
        origline.set_visible(visible)
        # Change the alpha on the line in the legend so we can see what lines
        # have been toggled.
        legline.set_alpha(1.0 if visible else 0.2)
        fig.canvas.draw()

    fig.canvas.mpl_connect('pick_event', on_pick)
#     plt.show()
        

def plotPressure(start_date, end_date=None):
    df_datetimes_all, df_pressures_all = loadPressure(start_date, end_date)

    fig, ax = plt.subplots()
    
    text = "Pressures from " + start_date.strftime("%y-%m-%d") + " to " + end_date.strftime("%y-%m-%d")
    title = "Click on legend line to toggle line on/off" + "\n" + "\n".join(textwrap.wrap(text, 60))
    ax.set_title(title)
    ax.set_xlabel('Datetime')
    ax.set_ylabel('Pressure (mBar)')
    ax.grid()
    
    line1, = ax.plot(df_datetimes_all, df_pressures_all.iloc[:,0], label="P1")
    line2, = ax.plot(df_datetimes_all, df_pressures_all.iloc[:,1], label="P2")
    line3, = ax.plot(df_datetimes_all, df_pressures_all.iloc[:,2], label="P3")
    line4, = ax.plot(df_datetimes_all, df_pressures_all.iloc[:,3], label="P4")
    line5, = ax.plot(df_datetimes_all, df_pressures_all.iloc[:,4], label="P5")
    line6, = ax.plot(df_datetimes_all, df_pressures_all.iloc[:,5], label="P6")
    leg = ax.legend(fancybox=True, shadow=True)
    
    lines = [line1, line2, line3, line4, line5, line6]
    lined = {}  # Will map legend lines to original lines.
    for legline, origline in zip(leg.get_lines(), lines):
        legline.set_picker(True)  # Enable picking on the legend line.
        lined[legline] = origline
    
    def on_pick(event):
        # On the pick event, find the original line corresponding to the legend
        # proxy line, and toggle its visibility.
        legline = event.artist
        origline = lined[legline]
        visible = not origline.get_visible()
        origline.set_visible(visible)
        # Change the alpha on the line in the legend so we can see what lines
        # have been toggled.
        legline.set_alpha(1.0 if visible else 0.2)
        fig.canvas.draw()

    fig.canvas.mpl_connect('pick_event', on_pick)
#     plt.show()

def plotFlowmeter(start_date, end_date=None):
    df_datetimes_all, df_flowmeters_all = loadFlowmeter(start_date, end_date)

    fig, ax = plt.subplots()
    
    text = "Flowmeter from " + start_date.strftime("%y-%m-%d") + " to " + end_date.strftime("%y-%m-%d")
    title = "Click on legend line to toggle line on/off" + "\n" + "\n".join(textwrap.wrap(text, 60))
    ax.set_title(title)
    ax.set_xlabel('Datetime')
    ax.set_ylabel('Flowrate (mmol/s)')
    ax.grid()
    
    line1, = ax.plot(df_datetimes_all, df_flowmeters_all.iloc[:,0], label="Flowmeter")
    leg = ax.legend(fancybox=True, shadow=True)
    

# Plot BlueFors log

In [110]:
# Usage:
# set start_date and end_date. If only start_date is set, only data in start_date is plotted.
# Example: 
# start_date = '18-10-01'
# end_date = '18-10-10'
# plotTemperature(start_date, end_date)
# plotTemperature(start_date)


start_date = date(22,1,2)
end_date   = date(22, 10,8)

plotTemperature(start_date, end_date)
# plotPressure(start_date, end_date)
# plotFlowmeter(start_date, end_date)
